In [1]:
!pip install git+https://github.com/facebookresearch/detectron2.git -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.2 MB/s eta 0:00:00


In [2]:
# COMMON LIBRARIES
import os
import cv2
import json
import torch

from datetime import datetime

# DATA SET PREPARATION AND LOADING
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# VISUALIZATION
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# CONFIGURATION
from detectron2 import model_zoo
from detectron2.config import get_cfg

# EVALUATION
from detectron2.engine import DefaultPredictor

# TRAINING
from detectron2.engine import DefaultTrainer

In [3]:
!pip install roboflow -q

from roboflow import Roboflow
rf = Roboflow(api_key="p7ll5rTRQYfRRG5zTZYK")
project = rf.workspace("amodic2c").project("paddy_weed")
version = project.version(2)
dataset = version.download("coco-segmentation")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to paddy_weed-2 in coco-segmentation:: 100%|██████████| 27951/27951 [00:11<00:00, 2417.79it/s]


In [4]:
dataset_path = dataset.location

In [6]:
import json

annotation_file = "/content/paddy_weed-2/train/_annotations.coco.json"

with open(annotation_file, "r") as f:
    data = json.load(f)

category_ids = {c["id"]: c["name"] for c in data["categories"]}
print("Category Mapping:", category_ids)


Category Mapping: {0: 'weed', 1: 'Paddy', 2: 'Weed'}


In [7]:
import json

def count_instances(annotation_file):
    with open(annotation_file, 'r') as f:
        coco_data = json.load(f)

    category_counts = {}

    for ann in coco_data["annotations"]:
        cat_id = ann["category_id"]
        category_counts[cat_id] = category_counts.get(cat_id, 0) + 1

    # Print instance counts per category
    print("Instance counts per category:")
    for cat_id, count in category_counts.items():
        category_name = next((c["name"] for c in coco_data["categories"] if c["id"] == cat_id), f"Unknown ({cat_id})")
        print(f"{category_name} (ID {cat_id}): {count} instances")

    print(f"\nTotal annotations: {len(coco_data['annotations'])}")

# Usage
count_instances("/content/paddy_weed-2/train/_annotations.coco.json")


Instance counts per category:
Paddy (ID 1): 20508 instances
Weed (ID 2): 31356 instances

Total annotations: 51864


In [16]:
import json

def fix_annotations(input_file, output_file):
    with open(input_file, "r") as f:
        coco_data = json.load(f)

    # Remove Paddy (ID 1)
    filtered_annotations = [ann for ann in coco_data["annotations"] if ann["category_id"] != 1]

    # Merge Weed categories (0 & 2) into a single ID 1
    for ann in filtered_annotations:
        if ann["category_id"] in [0, 2]:
            ann["category_id"] = 1  # Assign new ID for weed

    # Update categories to keep only one "Weed"
    coco_data["categories"] = [{"id": 1, "name": "weed"}]

    # Update dataset
    coco_data["annotations"] = filtered_annotations

    # Save new annotation file
    with open(output_file, "w") as f:
        json.dump(coco_data, f, indent=4)

    print(f"✅ New annotation file saved as: {output_file}")
    print(f"🔹 Remaining Weed Instances: {len(filtered_annotations)}")

# Usage
fix_annotations("/content/paddy_weed-2/valid/_annotations.coco.json",
                "/content/paddy_weed-2/valid/_annotations_fixed.coco.json")


✅ New annotation file saved as: /content/paddy_weed-2/valid/_annotations_fixed.coco.json
🔹 Remaining Weed Instances: 1259


In [9]:
DATA_SET_NAME = dataset_path.split("/")[-1].replace(" ", "-")
ANNOTATIONS_FILE_NAME = "_annotations_fixed.coco.json"

In [10]:
# TRAIN SET
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset_path, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset_path, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME,
    metadata={},
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH,
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

# TEST SET
TEST_DATA_SET_NAME = f"{DATA_SET_NAME}-test"
TEST_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset_path, "test")
TEST_DATA_SET_ANN_FILE_PATH = os.path.join(dataset_path, "test", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TEST_DATA_SET_NAME,
    metadata={},
    json_file=TEST_DATA_SET_ANN_FILE_PATH,
    image_root=TEST_DATA_SET_IMAGES_DIR_PATH
)

# VALID SET
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset_path, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset_path, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=VALID_DATA_SET_NAME,
    metadata={},
    json_file=VALID_DATA_SET_ANN_FILE_PATH,
    image_root=VALID_DATA_SET_IMAGES_DIR_PATH
)

In [14]:
TEST_DATA_SET_ANN_FILE_PATH

'/content/paddy_weed-2/test/_annotations_fixed.coco.json'

In [11]:
[
    data_set
    for data_set
    in MetadataCatalog.list()
    if data_set.startswith(DATA_SET_NAME)
]

['paddy_weed-2-train', 'paddy_weed-2-test', 'paddy_weed-2-valid']

In [23]:
from detectron2.engine import HookBase, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.solver import build_lr_scheduler
from detectron2 import model_zoo
import torch
import json

class LRSchedulerWithEarlyStopping(HookBase):
    def __init__(self, patience=5):
        self.patience = patience
        self.best_loss = float('inf')
        self.waiting = 0

    def after_step(self):
        stats = self.trainer.storage.latest()
        current_loss = stats.get("total_loss", float("inf"))

        if isinstance(current_loss, (list, tuple)):
            current_loss = current_loss[0]

        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.waiting = 0
        else:
            self.waiting += 1

        if self.waiting >= self.patience and self.trainer.scheduler is not None:
            self.trainer.scheduler.step()
            self.waiting = 0

class WeedTrainer(DefaultTrainer):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.scheduler = build_lr_scheduler(cfg, self.optimizer)

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.append(LRSchedulerWithEarlyStopping(patience=5))
        return hooks


def get_optimized_weed_detection_config():
    cfg = get_cfg()
    ARCHITECTURE = "mask_rcnn_X_101_32x8d_FPN_3x"
    CONFIG_FILE_PATH = f"COCO-InstanceSegmentation/{ARCHITECTURE}.yaml"
    cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))

    cfg.OUTPUT_DIR = "/content/output"

    # Dataset Configuration
    cfg.DATASETS.TRAIN = ("paddy_weed-2-train",)
    cfg.DATASETS.TEST = ("paddy_weed-2-test",)


    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.MASK_ON = True
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Optimized Training Hyperparameters
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.BASE_LR = 0.0003
    cfg.SOLVER.MAX_ITER = 15000
    cfg.SOLVER.STEPS = (20000, 25000, 28000)
    cfg.SOLVER.GAMMA = 0.1
    cfg.SOLVER.WARMUP_ITERS = 2000
    cfg.SOLVER.WEIGHT_DECAY = 0.00025
    cfg.SOLVER.MOMENTUM = 0.9
    cfg.SOLVER.CLIP_GRADIENTS.ENABLED = True
    cfg.SOLVER.CHECKPOINT_PERIOD = 1000

    # ROI Head Configuration
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
    cfg.MODEL.ROI_HEADS.POSITIVE_FRACTION = 0.25
    cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.3
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
    cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.3, 0.5, 0.7]
    cfg.MODEL.ROI_HEADS.IOU_LABELS = [0, 1, 1, 1]

    # RPN Configuration
    cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 256
    cfg.MODEL.RPN.POSITIVE_FRACTION = 0.5
    cfg.MODEL.RPN.PRE_NMS_TOPK_TRAIN = 6000
    cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 1000
    cfg.MODEL.RPN.PRE_NMS_TOPK_TEST = 3000
    cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 500
    cfg.MODEL.RPN.NMS_THRESH = 0.7

    # Anchor Generator
    cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16, 32, 64, 128, 256]]
    cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.25, 0.5, 1.0, 2.0]]

    # Input Configuration
    cfg.INPUT.MIN_SIZE_TRAIN = (640, 672, 704, 736, 768, 800)
    cfg.INPUT.MAX_SIZE_TRAIN = 1333
    cfg.INPUT.MIN_SIZE_TEST = 800
    cfg.INPUT.MAX_SIZE_TEST = 1333
    cfg.INPUT.MASK_FORMAT = "bitmask"

    # Data Augmentation
    cfg.INPUT.RANDOM_FLIP = "horizontal"
    cfg.INPUT.BRIGHTNESS = 1.0
    cfg.INPUT.CONTRAST = 1.0
    cfg.INPUT.SATURATION = 1.0
    cfg.INPUT.ROTATION = [-15, 15]

    # Dataloader
    cfg.DATALOADER.NUM_WORKERS = 4
    cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
    cfg.DATALOADER.ASPECT_RATIO_GROUPING = True

    # Evaluation
    cfg.TEST.EVAL_PERIOD = 1000
    cfg.TEST.DETECTIONS_PER_IMAGE = 100
    cfg.TEST.PRECISE_BN.ENABLED = True
    cfg.TEST.PRECISE_BN.NUM_ITER = 200

    # Test-Time Augmentation
    cfg.TEST.AUG.ENABLED = True
    cfg.TEST.AUG.MIN_SIZES = (600, 700, 800, 900, 1000)
    cfg.TEST.AUG.MAX_SIZE = 1400
    cfg.TEST.AUG.FLIP = True

    # Mixed Precision Training
    cfg.SOLVER.AMP.ENABLED = False

    return cfg


cfg = get_optimized_weed_detection_config()
trainer = WeedTrainer(cfg)
trainer.resume_or_load(resume=True)  # Resume from last checkpoint
trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 18.12 MiB is free. Process 16220 has 14.72 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 7.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [18]:
f = open('config.yaml', 'w')
f.write(cfg.dump())
f.close()